In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import sncosmo
import tdastro

# Append the path to the test directory so we can import run_snia_end2end
test_path = tdastro._TDASTRO_TEST_DIR
sys.path.append(str(test_path.resolve()))

from sources.test_snia import run_snia_end2end

### Create the data we will use for this test

Load a sample opsim file (opsim_shorten.db) from the test's data directory and use the `oversample_opsim()` function to sample every 0.01 days from MJD=61406.0 to MJD=61771.0.

In [ ]:
from tdastro.astro_utils.opsim import OpSim, oversample_opsim

opsim_name = os.path.join(tdastro._TDASTRO_TEST_DATA_DIR, "opsim_shorten.db")
base_opsim = OpSim.from_db(opsim_name)
oversampled_observations = oversample_opsim(
    base_opsim,
    pointing=(0.0, 0.0),
    search_radius=180.0,
    delta_t=0.01,
    time_range=(61406.0, 61771.0),
    bands=None,
    strategy="darkest_sky",
)

### define the source and observation

### Run the test 
Run the end to end test using the `run_snia_end2end()` to generate 20 samples.

TODO: use timeit or profiler here (repeat with a different nsample)

In [ ]:
passbands_dir = os.path.join(tdastro._TDASTRO_TEST_DATA_DIR, "passbands")
res, passbands = run_snia_end2end(
    oversampled_observations,
    passbands_dir=passbands_dir,
    nsample=20,
)

print(f"Produced {len(res)} samples.")

### examine model parameters

In [ ]:
hostmass = [x["parameter_values"]["hostmass"] for x in res]
x1 = [x["parameter_values"]["x1"] for x in res]
x0 = [x["parameter_values"]["x0"] for x in res]
c = [x["parameter_values"]["c"] for x in res]
z = [x["parameter_values"]["redshift"] for x in res]
t0 = [x["parameter_values"]["t0"] for x in res]
distmod = [x["parameter_values"]["distmod"] for x in res]

In [ ]:
plt.hist(hostmass)

In [ ]:
plt.hist(x1)
print(np.std(x1))

In [ ]:
plt.hist(c)

In [ ]:
plt.scatter(hostmass, x1)

In [ ]:
plt.hist(x0)

In [ ]:
plt.hist(z)

### examine physical effects

In [ ]:
# cosmo = FlatLambdaCDM(H0=73, Om0=0.3)
# distmod = cosmo.distmod(z).value

#    x0 = np.power(10., -0.4 * (distmod - alpha * x1 + beta * c + m_abs))
#  -2.5*log10(x0) = distmod -alpha*x1 + beta*c + m_abs

mb = -2.5 * np.log10(x0)
print(np.std(mb - distmod))

plt.scatter(z, mb - distmod)
plt.show()

alpha = 0.14
beta = 3.1
mu = np.array(mb) + alpha * np.array(x1) - beta * np.array(c) + 19.3
print(np.std(mu - distmod))

plt.scatter(z, mu - distmod)

plt.show()

In [ ]:
plt.scatter(z, mb)
plt.show()
plt.scatter(z, mu)
plt.show()

In [ ]:
plt.scatter(hostmass, mu - distmod)
plt.show()

### examine the source model

In [ ]:
for i in range(0, 3):
    if res[i] is None:
        continue
    plt.plot(res[i]["wavelengths_rest"] * (1 + z[i]), res[i]["flux_flam"][0], color="r")
    saltpars = {"x0": x0[i], "x1": x1[i], "c": c[i], "z": z[i], "t0": t0[i]}
    model = sncosmo.Model("salt3")
    model.update(saltpars)
    print(saltpars)
    print(model.parameters)
    print(res[i]["times"] - t0[i])
    wave = res[i]["wavelengths_rest"] * (1 + z[i])
    plt.plot(wave, model.flux(res[i]["times"][0], wave), color="g")
    plt.show()

### examine the light curves

In [ ]:
colors = "green", "red"
mag_zp = 31.4
for i in range(0, 3):
    _fig, (ax_f, ax_m) = plt.subplots(2, figsize=(5, 8))
    for f, color in zip("gr", colors):
        band_name = f"LSST_{f}"
        band_idx = res[i]["filters"] == band_name
        t = res[i]["times"][band_idx]
        flux_perfect = res[i]["bandfluxes_perfect"][band_idx]
        flux = res[i]["bandfluxes"][band_idx]
        flux_err = res[i]["bandfluxes_error"][band_idx]
        mag = mag_zp - 2.5 * np.log10(flux)
        mag_plus = mag_zp - 2.5 * np.log10(flux - flux_err)
        mag_minus = mag_zp - 2.5 * np.log10(flux + flux_err)

        ax_f.plot(t, flux_perfect, "-", label=f, color=color, alpha=0.6, lw=2)
        ax_f.fill_between(t, flux - flux_err, flux + flux_err, color="black", alpha=0.3)

        # ax_m.scatter(t, mag, label=f, marker='x', s=8, color=color, lw=2)
        # ax_m.errorbar(t, mag_minus, max_plus, ls='', color=color)
        ax_m.fill_between(t, mag_minus, mag_plus, color=color, alpha=0.6)

        saltpars = {"x0": x0[i], "x1": x1[i], "c": c[i], "z": z[i], "t0": t0[i]}
        model = sncosmo.Model("salt3")
        model.update(saltpars)
        print(saltpars)

        sncosmo_band = sncosmo.Bandpass(
            *passbands.passbands[band_name].processed_transmission_table.T, name=band_name
        )
        flux_sncosmo = model.bandflux(sncosmo_band, t, zpsys="ab", zp=mag_zp)
        ax_f.plot(t, flux_sncosmo, "--", label=f, color=color)
    ax_f.set_xlabel("MJD")
    ax_f.set_ylabel("Flux, nJy")
    ax_m.set_xlabel("MJD")
    ax_m.set_ylabel("mag")
    ax_m.invert_yaxis()
    plt.legend()
    plt.show()

TODO: 
now we have lsst-like LCs and their true parameters
we should make idealized LCs with the same true parameters and no observational effects, just insanely high cadence opsim, etc.
Let's fit each set with scipy.optimize, then plot differences relative to truth (should be unbiased for perfect data and noisy for realistic)

TODO: fit population parameters from noisy data ensemble